## Arcene

<div class="alert alert-block alert-danger">
Data Import
</div>

Train

In [6]:
import pandas as pd
data_arcene_train=pd.read_table("arcene_train.data",sep="\s+",header=None)
labels_arcene_train=pd.read_table("arcene_train.labels",sep="\s+",header=None)
data_arcene_valid=pd.read_table("arcene_valid.data",sep="\s+",header=None)
labels_arcene_valid=pd.read_table("arcene_valid.labels",sep="\s+",header=None)
data_arcene_train.head()

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0,71,0,95,0,538,404,20,0,0,...,255,570,86,0,36,0,80,0,0,524
1,0,41,82,165,60,554,379,0,71,0,...,213,605,69,7,473,0,57,0,284,423
2,0,0,1,40,0,451,402,0,0,0,...,235,593,28,0,24,0,90,0,34,508
3,0,56,44,275,14,511,470,0,0,0,...,91,600,0,26,86,0,102,0,0,469
4,105,0,141,348,0,268,329,0,0,1,...,813,0,0,0,0,190,301,0,0,354


In [7]:
data_arcene_train.shape

(100, 10000)

Test

In [8]:
data_arcene_test=pd.read_table("arcene_test.data",sep="\s+",header=None)
data_arcene_test.shape

(700, 10000)

<div class="alert alert-block alert-danger">
Use of All features
</div>

Classifier

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsClassifier

In [20]:
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
clf_arcene=Ridge(alpha=10)
clf_arcene.fit(data_gisette_train,labels_gisette_train)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

<div class="alert alert-block alert-danger">
Ridge features selection
</div>


In [10]:
clf_arcene=Ridge(alpha=1)
clf_arcene.fit(data_arcene_train,labels_arcene_train)
features_Ridge_arcene=[]
for i in range(len(clf_arcene.coef_[0])):
    if clf_arcene.coef_[0][i]>0:
        features_Ridge_arcene.append(i)

In [12]:
len(features_Ridge_arcene)*100/data_arcene_train.shape[1]

47.2

In [15]:
import numpy as np
CLF=RandomForestClassifier(n_estimators=1000)
CLF.fit(data_arcene_train[features_Ridge_arcene],labels_arcene_train)
np.savetxt("Ridge/arcene_test.predict",CLF.predict_proba(data_arcene_test[features_Ridge_arcene])[:,1],fmt="%f")

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


<div class="alert alert-block alert-danger">
Lasso features selection
</div>

In [21]:
clf_arcene=Lasso(alpha=0.001)
clf_arcene.fit(data_arcene_train,labels_arcene_train)
features_Lasso_arcene=[]
for i in range(len(clf_arcene.coef_)):
    if clf_arcene.coef_[i]>0:
        features_Lasso_arcene.append(i)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [22]:
len(features_Lasso_arcene)*100/data_arcene_train.shape[1]

4.83

In [24]:
CLF=RandomForestClassifier(n_estimators=1000)
CLF.fit(data_arcene_train[features_Lasso_arcene],labels_arcene_train)
np.savetxt("Lasso/arcene_test.predict",CLF.predict_proba(data_arcene_test[features_Lasso_arcene])[:,1],fmt="%f")

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


<div class="alert alert-block alert-danger">
RandomForest feature selection
</div>

In [26]:
from sklearn.feature_selection import SelectFromModel
sel_arcene = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel_arcene.fit(data_arcene_train,labels_arcene_train)
selected_feat_arcene= data_arcene_train.columns[(sel_arcene.get_support())]
data_arcene_train_reduced=data_arcene_train[selected_feat_arcene]
data_arcene_valid_reduced=data_arcene_valid[selected_feat_arcene]
clf_arcene_reduced=RandomForestClassifier()
clf_arcene_reduced.fit(data_arcene_train_reduced,labels_arcene_train)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
np.savetxt("RandomForest/arcene_test.predict",clf_arcene_reduced.predict_proba(data_arcene_test[selected_feat_arcene])[:,1],fmt="%f")

In [28]:
len(selected_feat_arcene)*100/data_arcene_train.shape[1]

8.27

## Gisette

<div class="alert alert-block alert-danger">
Data Import
</div>

Train

In [29]:
data_gisette_train=pd.read_table("gisette_train.data",sep="\s+",header=None)
labels_gisette_train=pd.read_table("gisette_train.labels",sep="\s+",header=None)
data_gisette_train.head()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,550,0,495,0,0,0,0,976,0,0,...,0,0,0,991,991,0,0,0,0,983
1,0,0,0,0,0,0,0,976,0,0,...,0,475,0,991,0,0,991,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,742,0,0,0,0,684,0,956,...,0,0,0,0,0,0,674,0,0,838
4,0,0,0,0,0,0,0,608,0,979,...,991,0,0,828,0,0,0,0,0,0


In [30]:
data_gisette_train.shape

(6000, 5000)

Test

In [31]:
data_gisette_test=pd.read_table("gisette_test.data",sep="\s+",header=None)
data_gisette_test.shape

(6500, 5000)

<div class="alert alert-block alert-danger">
Use of all features
</div>

Classifier

In [32]:
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
clf_gisette=RandomForestClassifier()
clf_gisette.fit(data_gisette_train,labels_gisette_train)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [33]:
data_gisette_valid=pd.read_table("gisette_valid.data",sep="\s+",header=None)
labels_gisette_valid=pd.read_table("gisette_valid.labels",sep="\s+",header=None)
roc_auc_score(labels_gisette_valid,clf_gisette.predict_proba(data_gisette_valid)[:,1])

0.990332

<div class="alert alert-block alert-danger">
Ridge features selection
</div>

In [43]:
clf_gisette=Ridge(alpha=1)
clf_gisette.fit(data_gisette_train,labels_gisette_train)
features_Ridge_gisette=[]
for i in range(len(clf_gisette.coef_[0])):
    if clf_gisette.coef_[0][i]>0:
        features_Ridge_gisette.append(i)

In [44]:
len(features_Ridge_gisette)*100/data_gisette_train.shape[1]

48.4

In [45]:
CLF=RandomForestClassifier(n_estimators=1000)
CLF.fit(data_gisette_train[features_Ridge_gisette],labels_gisette_train)
np.savetxt("Ridge/gisette_test.predict",CLF.predict_proba(data_gisette_test[features_Ridge_gisette])[:,1],fmt="%f")

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


<div class="alert alert-block alert-danger">
Lasso features selection
</div>

In [46]:
clf_gisette=Lasso(alpha=1)
clf_gisette.fit(data_gisette_train,labels_gisette_train)
features_Lasso_gisette=[]
for i in range(len(clf_gisette.coef_)):
    if clf_gisette.coef_[i]>0:
        features_Lasso_gisette.append(i)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [47]:
len(features_Lasso_gisette)*100/data_gisette_train.shape[1]

10.28

In [48]:
CLF=RandomForestClassifier(n_estimators=1000)
CLF.fit(data_gisette_train[features_Lasso_gisette],labels_gisette_train)
np.savetxt("Lasso/gisette_test.predict",CLF.predict_proba(data_gisette_test[features_Lasso_gisette])[:,1],fmt="%f")

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


<div class="alert alert-block alert-danger">
RandomForest Feature selection
</div>

In [49]:
from sklearn.feature_selection import SelectFromModel
sel_gisette = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel_gisette.fit(data_gisette_train,labels_gisette_train)
selected_feat_gisette= data_gisette_train.columns[(sel_gisette.get_support())]
data_gisette_train_reduced=data_gisette_train[selected_feat_gisette]
data_gisette_valid_reduced=data_gisette_valid[selected_feat_gisette]
clf_gisette_reduced=RandomForestClassifier()
clf_gisette_reduced.fit(data_gisette_train_reduced,labels_gisette_train)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [50]:
np.savetxt("RandomForest/gisette_test.predict",clf_gisette_reduced.predict_proba(data_gisette_test[selected_feat_gisette])[:,1],fmt="%f")

In [51]:
len(selected_feat_gisette)*100/data_gisette_train.shape[1]

12.52

## Madelon

<div class="alert alert-block alert-danger">
Data Import
</div>

Train

In [52]:
data_madelon_train=pd.read_csv("madelon_train.data",sep="\s+",header=None)
labels_madelon_train=pd.read_table("madelon_train.labels",sep="\s+",header=None)
data_madelon_train.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,485,477,537,479,452,471,491,476,475,473,...,477,481,477,485,511,485,481,479,475,496
1,483,458,460,487,587,475,526,479,485,469,...,463,478,487,338,513,486,483,492,510,517
2,487,542,499,468,448,471,442,478,480,477,...,487,481,492,650,506,501,480,489,499,498
3,480,491,510,485,495,472,417,474,502,476,...,491,480,474,572,454,469,475,482,494,461
4,484,502,528,489,466,481,402,478,487,468,...,488,479,452,435,486,508,481,504,495,511


valid

In [53]:
data_madelon_valid=pd.read_table("madelon_valid.data",sep="\s+",header=None)
labels_madelon_valid=pd.read_table("madelon_valid.labels",sep="\s+",header=None)

Test

In [54]:
data_madelon_test=pd.read_table("madelon_test.data",sep="\s+",header=None)
data_madelon_test.shape

(1800, 500)

<div class="alert alert-block alert-danger">
Use of all features
</div>

Classifier

In [55]:
clf_madelon=RandomForestClassifier()
clf_madelon.fit(data_madelon_train,labels_madelon_train)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

<div class="alert alert-block alert-danger">
Ridge feature selection
</div>

In [61]:
clf_madelon=Ridge(alpha=1)
clf_madelon.fit(data_madelon_train,labels_madelon_train)
features_Ridge_madelon=[]
for i in range(len(clf_madelon.coef_[0])):
    if clf_madelon.coef_[0][i]>0:
        features_Ridge_madelon.append(i)

In [62]:
len(features_Ridge_madelon)*100/data_madelon_train.shape[1]

49.8

In [63]:
CLF=RandomForestClassifier(n_estimators=1000)
CLF.fit(data_madelon_train[features_Ridge_madelon],labels_madelon_train)
np.savetxt("Ridge/madelon_test.predict",CLF.predict_proba(data_madelon_test[features_Ridge_madelon])[:,1],fmt="%f")

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


<div class="alert alert-block alert-danger">
Lasso Feature selection
</div>

In [67]:
clf_madelon=Lasso(alpha=0.1)
clf_madelon.fit(data_madelon_train,labels_madelon_train)
features_Lasso_madelon=[]
for i in range(len(clf_madelon.coef_)):
    if clf_madelon.coef_[i]>0:
        features_Lasso_madelon.append(i)

In [68]:
len(features_Lasso_madelon)*100/data_madelon_train.shape[1]

34.2

In [69]:
CLF=RandomForestClassifier(n_estimators=1000)
CLF.fit(data_madelon_train[features_Lasso_madelon],labels_madelon_train)
np.savetxt("Lasso/madelon_test.predict",CLF.predict_proba(data_madelon_test[features_Lasso_madelon])[:,1],fmt="%f")

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


<div class="alert alert-block alert-danger">
RandomForest feature selection
</div>

In [70]:
from sklearn.feature_selection import SelectFromModel
sel_madelon = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel_madelon.fit(data_madelon_train,labels_madelon_train)
selected_feat_madelon= data_madelon_train.columns[(sel_madelon.get_support())]
data_madelon_train_reduced=data_madelon_train[selected_feat_madelon]
data_madelon_valid_reduced=data_madelon_valid[selected_feat_madelon]
clf_madelon_reduced=RandomForestClassifier()
clf_madelon_reduced.fit(data_madelon_train_reduced,labels_madelon_train)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [71]:
np.savetxt("RandomForest/madelon_test.predict",clf_madelon_reduced.predict_proba(data_madelon_test[selected_feat_madelon])[:,1],fmt="%f")

In [72]:
len(selected_feat_madelon)*100/data_madelon_train.shape[1]

27.4

## Dorothea

<div class="alert alert-block alert-danger">
Data Import
</div>

Train

In [73]:
import numpy as np
with open("dorothea_train.data","r") as f:
       content = f.read()
data_dorothea_train = np.zeros((800,100000))
for i,el in enumerate(content.split("\n")[:-1]):
    for column in el.strip().split():
        data_dorothea_train[i,np.int(column)-1] = 1
labels_dorothea_train=pd.read_table("dorothea_train.labels",sep="\s+",header=None)

Valid

In [74]:
with open("dorothea_valid.data","r") as f:
       content = f.read()
data_dorothea_valid = np.zeros((350,100000))
for i,el in enumerate(content.split("\n")[:-1]):
    for column in el.strip().split():
        data_dorothea_valid[i,np.int(column)-1] = 1
labels_dorothea_valid=pd.read_table("dorothea_valid.labels",sep="\s+",header=None)

Test

In [75]:
with open("dorothea_test.data","r") as f:
       content = f.read()
data_dorothea_test = np.zeros((800,100000))
for i,el in enumerate(content.split("\n")[:-1]):
    for column in el.strip().split():
        data_dorothea_test[i,np.int(column)-1] = 1

<div class="alert alert-block alert-danger">
Use of all features
</div>

Classifier

In [76]:
data_dorothea_train=pd.DataFrame(data_dorothea_train)
data_dorothea_valid=pd.DataFrame(data_dorothea_valid)
data_dorothea_test=pd.DataFrame(data_dorothea_test)
clf_dorothea=RandomForestClassifier()
clf_dorothea.fit(data_dorothea_train,labels_dorothea_train)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

<div class="alert alert-block alert-danger">
Ridge feature selection
</div>

In [77]:
clf_dorothea=Ridge(alpha=1)
clf_dorothea.fit(data_dorothea_train,labels_dorothea_train)
features_Ridge_dorothea=[]
for i in range(len(clf_dorothea.coef_[0])):
    if clf_dorothea.coef_[0][i]>0:
        features_Ridge_dorothea.append(i)

In [78]:
len(features_Ridge_dorothea)*100/data_dorothea_train.shape[1]

40.821

In [79]:
CLF=RandomForestClassifier(n_estimators=1000)
CLF.fit(data_dorothea_train[features_Ridge_dorothea],labels_dorothea_train)
np.savetxt("Ridge/dorothea_test.predict",CLF.predict_proba(data_dorothea_test[features_Ridge_dorothea])[:,1],fmt="%f")

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


<div class="alert alert-block alert-danger">
Lasso Feature selection
</div>

In [123]:
clf_dorothea=Lasso(alpha=0.00001)
clf_dorothea.fit(data_dorothea_train,labels_dorothea_train)
features_Lasso_dorothea=[]
for i in range(len(clf_dorothea.coef_)):
    if clf_dorothea.coef_[i]>0:
        features_Lasso_dorothea.append(i)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [124]:
len(features_Lasso_dorothea)*100/data_dorothea_train.shape[1]

0.806

In [125]:
CLF=RandomForestClassifier(n_estimators=1000)
CLF.fit(data_dorothea_train[features_Lasso_dorothea],labels_dorothea_train)
np.savetxt("Lasso/dorothea_test.predict",CLF.predict_proba(data_dorothea_test[features_Lasso_dorothea])[:,1],fmt="%f")

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


<div class="alert alert-block alert-danger">
Random Forest feature selection
</div>

In [101]:
from sklearn.feature_selection import SelectFromModel
sel_dorothea = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel_dorothea.fit(data_dorothea_train,labels_dorothea_train)
selected_feat_dorothea= data_dorothea_train.columns[(sel_dorothea.get_support())]
data_dorothea_train_reduced=data_dorothea_train[selected_feat_dorothea]
data_dorothea_valid_reduced=data_dorothea_valid[selected_feat_dorothea]
clf_dorothea_reduced=RandomForestClassifier()
clf_dorothea_reduced.fit(data_dorothea_train_reduced,labels_dorothea_train)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [102]:
np.savetxt("RandomForest/dorothea_test.predict",clf_dorothea_reduced.predict_proba(data_dorothea_test[selected_feat_dorothea])[:,1],fmt="%f")

In [103]:
len(selected_feat_dorothea)*100/data_dorothea_train.shape[1]

3.788

## Dexter

<div class="alert alert-block alert-danger">
Data Import
</div>

Train

In [104]:
import re
import numpy as np
import pandas as pd
with open("dexter_train.data","r") as f:
       content = f.read()
data_dexter_train = np.zeros((300,20000))

for i,el in enumerate(content.split("\n")[:-1]):
    for row in el.strip().split(" "):
        a,b = row.split(":")
        data_dexter_train[i,np.int(a)-1] = np.int(b)
labels_dexter_train=pd.read_table("dexter_train.labels",sep="\s+",header=None)

Valid

In [105]:
with open("dexter_valid.data","r") as f:
       content = f.read()
data_dexter_valid = np.zeros((300,20000))

for i,el in enumerate(content.split("\n")[:-1]):
    for row in el.strip().split(" "):
        a,b = row.split(":")
        data_dexter_valid[i,np.int(a)-1] = np.int(b)
labels_dexter_valid=pd.read_table("dexter_valid.labels",sep="\s+",header=None)

Test

In [106]:
with open("dexter_test.data","r") as f:
       content = f.read()
data_dexter_test = np.zeros((2000,20000))
k=0
for i,el in enumerate(content.split("\n")[:-1]):
    for row in el.strip().split(" "):
        a,b = row.split(":")
        data_dexter_test[i,np.int(a)-1] = np.int(b)
        

<div class="alert alert-block alert-danger">
Use of al features
</div>

Classifier

In [107]:
data_dexter_train=pd.DataFrame(data_dexter_train)
data_dexter_valid=pd.DataFrame(data_dexter_valid)
data_dexter_test=pd.DataFrame(data_dexter_test)
clf_dexter=RandomForestClassifier()
clf_dexter.fit(data_dexter_train,labels_dexter_train)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

<div class="alert alert-block alert-danger">
Ridge feature selection
</div>

In [108]:
clf_dexter=Ridge(alpha=1)
clf_dexter.fit(data_dexter_train,labels_dexter_train)
features_Ridge_dexter=[]
for i in range(len(clf_dexter.coef_[0])):
    if clf_dexter.coef_[0][i]>0:
        features_Ridge_dexter.append(i)

In [109]:
len(features_Ridge_dexter)*100/data_dexter_train.shape[1]

18.915

In [110]:
CLF=RandomForestClassifier(n_estimators=1000)
CLF.fit(data_dexter_train[features_Ridge_dexter],labels_dexter_train)
np.savetxt("Ridge/dexter_test.predict",CLF.predict_proba(data_dexter_test[features_Ridge_dexter])[:,1],fmt="%f")

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


<div class="alert alert-block alert-danger">
Lasso feature selection
</div>

In [119]:
clf_dexter=Lasso(alpha=0.00001)
clf_dexter.fit(data_dexter_train,labels_dexter_train)
features_Lasso_dexter=[]
for i in range(len(clf_dexter.coef_)):
    if clf_dexter.coef_[i]>0:
        features_Lasso_dexter.append(i)

In [120]:
len(features_Lasso_dexter)*100/data_dexter_train.shape[1]

6.875

In [127]:
CLF=RandomForestClassifier(n_estimators=1000)
CLF.fit(data_dexter_train[features_Lasso_dexter],labels_dexter_train)
np.savetxt("Lasso/dexter_test.predict",CLF.predict_proba(data_dexter_test[features_Lasso_dexter])[:,1],fmt="%f")

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


<div class="alert alert-block alert-danger">
Random Forest Feature selection
</div>

In [128]:
from sklearn.feature_selection import SelectFromModel
sel_dexter = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel_dexter.fit(data_dexter_train,labels_dexter_train)
selected_feat_dexter= data_dexter_train.columns[(sel_dexter.get_support())]
data_dexter_train_reduced=data_dexter_train[selected_feat_dexter]
data_dexter_valid_reduced=data_dexter_valid[selected_feat_dexter]
clf_dexter_reduced=RandomForestClassifier()
clf_dexter_reduced.fit(data_dexter_train_reduced,labels_dexter_train)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [129]:
len(selected_feat_dexter)*100/data_dexter_train.shape[1]

10.9

In [130]:
np.savetxt("RandomForest/dexter_test.predict",clf_dexter_reduced.predict_proba(data_dexter_test[selected_feat_dexter])[:,1],fmt="%f")